In [1]:
# # Import packages
import pandas as pd
import os, glob
import json
from sklearn.model_selection import train_test_split
import nltk
from os import getcwd
import random
import operator

In [2]:
#from bs4 import BeautifulSoup
from urllib.request import urlopen

In [3]:
# To extract the NEs
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

In [4]:
pip install newspaper3k 

Note: you may need to restart the kernel to use updated packages.


In [5]:
from newspaper import Article

In [6]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [7]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [8]:
#headlines_processed = []
#clean_news = []
text = []
title = []
count = 0

In [9]:
#path_covid_dataset = './dataset/16119_db21c91a1ab47385bb13773ed8238c31'
path_cnn_downloads = './dataset/cnn/downloads'
path_models = './models/'
path_outputs = './outputs/'
path_processed_data= './processed_data/'
getcwd()

'/home/ubuntu/w266-finalproject'

In [10]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",

                           "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",

                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",

                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",

                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",

                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",

                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",

                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",

                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",

                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",

                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",

                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",

                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",

                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",

                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",

                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",

                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",

                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",

                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",

                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",

                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",

                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",

                           "you're": "you are", "you've": "you have"}

In [11]:
import re
from nltk.corpus import stopwords

stop_words = stopwords.words('english')

def preprocess(text):
#    text = text.lower() # lowercase
    text = text.split() # convert have'nt -> have not
    for i in range(len(text)):
        word = text[i]
        if word in contraction_mapping:
            text[i] = contraction_mapping[word]
    #text = " ".join(text)
    text = " ".join(text)
    text = text.split()
    newtext = []
    for word in text:
        if word not in stop_words:
            newtext.append(word)
    text = " ".join(newtext)
    text = text.replace("'s",'') # convert your's -> your
    text = re.sub(r'\(.*\)','',text) # remove (words)
    text = re.sub(r'[^a-zA-Z0-9. ]','',text) # remove punctuations
    text = re.sub(r'\.',' . ',text)
    return text

In [12]:
tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")
# device = 0, cuda
#nlp_ner = pipeline("ner", model=model, tokenizer=tokenizer, device=0)
nlp_ner = pipeline("ner", model=model, tokenizer=tokenizer)

In [13]:
from collections import Counter
 
def remove_duplicates(input):
 
    # split input string separated by space
    input = input.split(" ")
 
    # now create dictionary using counter method
    # which will have strings as key and their
    # frequencies as value
    unique_words = Counter(input)
 
    # joins two adjacent elements in iterable way
    s = " ".join(unique_words.keys())
    return s

In [14]:
# example = "My name is Wolfgang and I live in Berlin, Saratoga, December 2019, Washington, Bill, Delhi, Modi Putin "

# ner_results = nlp(example)
# print(ner_results)


#for item in X_test:
    
def get_entities(article):    
    
    entities = []
    ner = []
    ner_results = nlp_ner(article)
    ner.append(ner_results)
    nes = []
    for ne in range(len(ner_results)):
        if ner_results[ne]['entity'] == 'B-PER' or ner_results[ne]['entity'] == 'I-PER' or ner_results[ne]['entity'] == 'B-LOC' or ner_results[ne]['entity'] == 'I-LOC' or ner_results[ne]['entity'] == 'I-ORG' or ner_results[ne]['entity'] == 'B-ORG' or ner_results[ne]['entity'] == 'I-MISC':
            nes.append(ner_results[ne]['word'])

    if len(nes) == 0:
        nes.append('')
    
    named_entities = remove_duplicates(' '.join(nes).replace(" ##", "").replace("##", ""))
    #named_entities = re.sub(r'\ .* \ ', ' ', named_entities)
    return named_entities

In [15]:
example = "My name is Wolfgang and I live in Berlin, Saratoga. Community December 2019. Washington, Bill. Delhi, Modi Putin "

# get_entities(example)

In [16]:
def get_ner_text(article, entities, max_sentence_percent):

    #print(entities)
    entities_array = entities.split()
    # returns a string with ner sentences

    sentences_array = []
    ner_sentences = []

    sentences = nltk.sent_tokenize(article)
    #print(sentences)
    sentence_dict = {}
    sentence_order_dict = {}
    for k, sentence in enumerate(sentences):
        sentence_order_dict[sentence] = k
    
    sentence_count = len(sentences)
    #print("sentence count:", sentence_count)
    
    max_sentence_count = max(1, int(sentence_count * max_sentence_percent))
    #for sentence in sentences:
    for k, sentence in enumerate(sentences): 
        count = 0
        for word in entities_array:
            if word in sentence:
                count += 1
        #print(count)
        sentence_dict[sentence] = count
    
    sorted_sd = sorted(sentence_dict.items(), key=operator.itemgetter(1), reverse=True)

    for item in sorted_sd:
        ner_sentences.append(''.join(item[0]))
        #print(item[0], item[1])
    
    sentence_dict = {}
                
    for sentence in ner_sentences[:max_sentence_count]:
        sentence_dict[sentence] = sentence_order_dict[sentence]
        
    sorted_sd = sorted(sentence_dict.items(), key=operator.itemgetter(1), reverse=False)

    ner_sentences = []
    for item in sorted_sd:
        ner_sentences.append(''.join(item[0]))

    ner_sentences = ' '.join(ner_sentences)
            
    if len(ner_sentences) == 0:
        for item in sentences:
            len_sentences = min(max_sentence_count, len(sentences))
            ner_sentences = ' '.join(sentences[:len_sentences])
    
    return ner_sentences

In [17]:

# sentences = nltk.sent_tokenize(example)
# len_sentences = min(3, len(sentences))
# s3 = ' '.join(sentences[:len_sentences])
# s1 = ' '.join(sentences[:1])

# print(len(sentences))

In [18]:
import codecs
import io
import time


text = []
title = []
count = 0

X = []
Y = []
Z = []
W = []

start = time.time()

path_cnn_downloads = './dataset/cnn/downloads'
url =  'https://www.google.com/'

filenames = glob.glob(os.path.join(path_cnn_downloads, '*.html'))
#filenames = filenames[0:100]
# Number of files 92579

from joblib import Parallel, delayed
article = Article(url="%s" % (url), language='en')

def process_article(filename):
    article_text = ""
    text_ner = ""
    article_title = ""
    article_keywords = ""
    article_summary = ""
    entities = ""
    s1 = ""
    s3 = ""
    
    max_ner_percent = 0.5
    
    try:
        with io.open(filename, mode="r", encoding="unicode_escape") as f:
            html = f.read()
            html = html.encode('utf-8').strip()
            #article = Article(url="%s" % (url), language='en')
            article.download(input_html=html)
            article.parse()
            article.nlp()
            article_text = preprocess(article.text.strip())
            #article_title = article.title.encode('utf-8').strip()
            article_title = preprocess(article.title.strip())
            article_keywords = ' '.join([str(item) for item in article.keywords])
            article_summary = preprocess(article.summary.strip())
            entities = get_entities(article_text)
            text_ner = get_ner_text(article_text, entities, max_ner_percent)
            sentences = nltk.sent_tokenize(article_text)
            len_sentences = min(3, len(sentences))
            s3 = ' '.join(sentences[:len_sentences])
            s1 = ' '.join(sentences[:1])
    
            f.close()
    except Exception as error:
        f.close()
        print("Exception occurred: ", error)
        return article_text, article_title, article_keywords, article_summary, text_ner, entities, s1, s3
    return article_text, article_title, article_keywords, article_summary, text_ner, entities, s1, s3

S = Parallel(n_jobs=6)(delayed(process_article)(filename) for filename in filenames)

S = list(filter(None, S))

print("Response time (m): ", round((time.time() - start)/60, 2))


/home/ubuntu/anaconda3/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Response time (m):  1124.06


In [19]:
len(S)

92579

In [20]:
S_backup = S
text = []
headlines = []
keywords = []
summary_art = []
text_ner = []
entities = []
s1 = []
s3 = []

S_ = []

for i in range(len(S)):
    if len(S[i][0]) == 0 or len(S[i][1]) == 0 or len(S[i][2]) == 0 or len(S[i][3]) == 0 or len(S[i][4]) == 0 or len(S[i][5]) == 0:
        continue
    else:
        text.append(S[i][0])
        headlines.append(S[i][1])
        keywords.append(S[i][2])
        summary_art.append(S[i][3])
        text_ner.append(S[i][4])
        entities.append(S[i][5])
        s1.append(S[i][6])
        s3.append(S[i][7])
        S_.append(S[i])

S = S_

In [21]:
S[0][7]

'  Cristiano Ronaldo grabbed sixth hattrick season Real Madrid went 10 points clear archrivals Barcelona Spain 42 home win fourthplaced Levante . Sunday victory left Real driving seat win La Liga title following champions Barca 32 defeat Osasuna Saturday . Portugal star Ronaldo took league tally 27 goals season another superb display Gustavo Cabral gave Levante surprise early lead Bernabeu .'

In [22]:
count = 0
for i in range(len(S)):
    if len(S[i][0]) == 0 or len(S[i][1]) == 0 or len(S[i][2]) == 0 or len(S[i][3]) ==0 or len(S[i][4]) == 0 or len(S[i][4]) == 0:
        count += 1

print(count)

0


In [23]:
S[0][1]

'Ronaldo hattrick lifts Real Madrid 10 points clear Spain'

In [24]:
S[0][1]

'Ronaldo hattrick lifts Real Madrid 10 points clear Spain'

In [25]:
S[0][2]

'took victory league win real valencia ronaldo madrid title lifts hattrick spain second won clear points'

In [26]:
S[0][3]

'  Cristiano Ronaldo grabbed sixth hattrick season Real Madrid went 10 points clear archrivals Barcelona Spain 42 home win fourthplaced Levante .  Sunday victory left Real driving seat win La Liga title following champions Barca 32 defeat Osasuna Saturday .  Portugal star Ronaldo took league tally 27 goals season another superb display Gustavo Cabral gave Levante surprise early lead Bernabeu .  The mathematics tell us consider league won necessary get another 39 points 13 wins told AFP .  Substitute Jonas Goncalves scored twice injury time Valencia moved within eight points Barcelona Gijon remained second bottom . '

In [27]:
len(S)

91600

In [28]:
S_train, S_test = train_test_split(S, test_size=0.2)

In [29]:
S_valid, S_test = train_test_split(S_test, test_size=0.5)

In [30]:
text_train = []
headlines_train = []
keywords_train = []
summary_art_train = []
text_ner_train = []
entities_train = []
s1_train = []
s3_train = []

text_valid = []
headlines_valid = []
keywords_valid = []
summary_art_valid = []
text_ner_valid = []
entities_valid = []
s1_valid = []
s3_valid = []

text_test = []
headlines_test = []
keywords_test = []
summary_art_test = []
text_ner_test = []
entities_test = []
s1_test = []
s3_test = []

In [31]:
for i in range(len(S_train)):  
        text_train.append(S_train[i][0])
        headlines_train.append(S_train[i][1])
        keywords_train.append(S_train[i][2])
        summary_art_train.append(S_train[i][3])
        text_ner_train.append(S_train[i][4])
        entities_train.append(S_train[i][5])
        s1_train.append(S_train[i][6])
        s3_train.append(S_train[i][7])

In [32]:
for i in range(len(S_valid)):  
        text_valid.append(S_valid[i][0])
        headlines_valid.append(S_valid[i][1])
        keywords_valid.append(S_valid[i][2])
        summary_art_valid.append(S_valid[i][3])
        text_ner_valid.append(S_valid[i][4])
        entities_valid.append(S_valid[i][5])
        s1_valid.append(S_valid[i][6])
        s3_valid.append(S_valid[i][7])
        
for i in range(len(S_test)):  
        text_test.append(S_test[i][0])
        headlines_test.append(S_test[i][1])
        keywords_test.append(S_test[i][2])
        summary_art_test.append(S_test[i][3])
        text_ner_test.append(S_test[i][4])
        entities_test.append(S_test[i][5])
        s1_test.append(S_test[i][6])
        s3_test.append(S_test[i][7])

In [33]:
print(len(text_train), len(headlines_train), len(text_valid), len(headlines_valid), len(text_test), len(headlines_test))

73280 73280 9160 9160 9160 9160


In [34]:

dict_train = {
    'text' : text_train,
    'headlines' : headlines_train,
    'keywords' : keywords_train,
    'summary_art' : summary_art_train,
    'text_ner' : text_ner_train,
    'entities' : entities_train,
    's1' : s1_train,
    's3' : s3_train
}

list_to_json_array = json.dumps(dict_train)

jsonString = json.dumps(dict_train)
jsonFile = open(os.path.join(path_processed_data, 'cnn_train_v3.json'), "w")
jsonFile.write(jsonString)
jsonFile.close()



In [35]:
dict_valid = {
    'text' : text_valid,
    'headlines' : headlines_valid,
    'keywords' : keywords_valid,
    'summary_art' : summary_art_valid,
    'text_ner' : text_ner_valid,
    'entities' : entities_valid,
    's1' : s1_valid,
    's3' : s3_valid
}

list_to_json_array = json.dumps(dict_valid)

jsonString = json.dumps(dict_valid)
jsonFile = open(os.path.join(path_processed_data, 'cnn_valid_v3.json'), "w")
jsonFile.write(jsonString)
jsonFile.close()

In [36]:
# dict_test = {
#     'text' : text_test,
#     'headlines' : headlines_test,
#     'keywords' : keywords_test,
#     'summary_art' : summary_art_test,
#     'text_ner' : text_ner_test,
#     'entities' : entities_test,
#     's1' : s1_test,
#     's3' : s3_test

# }

# list_to_json_array = json.dumps(dict_test)

# jsonString = json.dumps(dict_vtest)
# jsonFile = open(os.path.join(path_processed_data, 'cnn_test_v3.json'), "w")
# jsonFile.write(jsonString)
# jsonFile.close()

In [37]:
# Another approach to get sentence tokens
# S1s = []
# S3s = []
# for item in X_test:
#     sentences = re.split(r'(?<=[.:;])\s', item)
#     len_sentences = min(3, len(sentences))
#     S3s.append(' '.join(sentences[:len_sentences]))
#     S1s.append(' '.join(sentences[:1]))

# print(len(S1s))

In [38]:
size = len(text_test)
print(size)

9160


In [39]:

size = len(text_test)

# S1s = [''] * size
# S3s = [''] * size
S2 = [''] * size
S3 = [''] * size
S4 = [''] * size
S5 = [''] * size
P0 = [''] * size
P1 = [''] * size
P2 = [''] * size
P3 = [''] * size
P4 = [''] * size
P5 = [''] * size
P6 = [''] * size
P6 = [''] * size
P7 = [''] * size

dict_test = {
    'text' : text_test,
    'headlines' : headlines_test,
    'keywords' : keywords_test,
    'summary_art' : summary_art_test,
    'entities' : entities_test,
    'text_ner' : text_ner_test,
    'sentence_1s' : s1_test,
    'sentence_3s' : s3_test,
    'summary_ext' : S2,
    'summary_abs' : S3,
    'summary_extabs' : S4,
    'summary_ner' : S5,
    'predict_1s' : P0,
    'predict_3s' : P1,
    'predict_text' : P2,
    'predict_ext' : P3,
    'predict_abs' : P4,
    'predict_extabs' : P5,
    'predict_ner' : P6,
    'predict_t5' : P7
}

df_test = pd.DataFrame(dict_test)

jsonString = json.dumps(dict_test)
jsonFile = open(os.path.join(path_processed_data, 'cnn_test_v3.json'), "w")
jsonFile.write(jsonString)
jsonFile.close()

In [40]:
df_test

,text,headlines,keywords,summary_art,entities,text_ner,sentence_1s,sentence_3s,summary_ext,summary_abs,summary_extabs,summary_ner,predict_1s,predict_3s,predict_text,predict_ext,predict_abs,predict_extabs,predict_ner,predict_t5
0,Story highlights Drummer Lee Riggers Rigby 25 ...,Soldier slain London street machine gunner dru...,street fusiliers member royal soldiers soldier...,Drummer Lee Rigby 25 part Regimental Recruitin...,Lee Riggers R Royal Palaces Jack London Afghan...,Story highlights Drummer Lee Riggers Rigby 25 ...,Story highlights Drummer Lee Riggers Rigby 25 ...,Story highlights Drummer Lee Riggers Rigby 25 ...,,,,,,,,,,,,
1,New York Officials investigating reports two...,New York probes report EMTs ignored woman died,emergency department woman office report spoke...,New York Officials investigating reports two...,New York Eutisha Renni Au Bon Pain Brooklyn Lo...,New York Officials investigating reports two...,New York Officials investigating reports two...,New York Officials investigating reports two...,,,,,,,,,,,,
2,Editor note Share volcano photos CNN iReport ....,Erupt typical travel pattern,active island pattern erupt travel list lava t...,Photos Earth otherworldly landscapesHere list ...,CNN iReport Williams Antarctica,Editor note Share volcano photos CNN iReport ....,Editor note Share volcano photos CNN iReport .,Editor note Share volcano photos CNN iReport ....,,,,,,,,,,,,
3,An Ohio jury Monday begin sentencing phase c...,Sentencing phase begin convicted Ohio serial k...,ohio sowells killer women begin sowell cnn tol...,An Ohio jury Monday begin sentencing phase c...,Ohio Anthony So Cleveland Ryan Mid Cuyahoga Co...,An Ohio jury Monday begin sentencing phase c...,An Ohio jury Monday begin sentencing phase c...,An Ohio jury Monday begin sentencing phase c...,,,,,,,,,,,,
4,Update In July 2014 Breanna Bond 12 competed...,Girl loses 65 pounds fight childhood obesity,family childhood kids eating food parents poun...,Update In July 2014 Breanna Bond 12 competed...,Breanna Bond Olympics Heidi B Centers Disease ...,Update In July 2014 Breanna Bond 12 competed...,Update In July 2014 Breanna Bond 12 competed...,Update In July 2014 Breanna Bond 12 competed...,,,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9155,outside experts help us establish whether thi...,What evidence Syrian chemical weapons attack,intelligence chemical syrian whats evidence us...,And Syrian President Bashar alAssad regime say...,Day Damascus France Germany BND Gerhard Schind...,outside experts help us establish whether thi...,outside experts help us establish whether thi...,outside experts help us establish whether thi...,,,,,,,,,,,,
9156,West Texas Since little boys growing West Te...,A town copes lives lost Texas blast,lives west school texas community students off...,West Texas Since little boys growing West Te...,West Texas Doug Robert S Fertilizer Co Kenneth...,West Texas Since little boys growing West Te...,West Texas Since little boys growing West Te...,West Texas Since little boys growing West Te...,,,,,,,,,,,,
9157,Government Accountability Office released rep...,Voluntary recall another compounding facility,voluntary facility compounding fda levels pati...,The FDA issued national voluntary sterileuse p...,Government Accountability Office FDA U . S,Government Accountability Office released rep...,Government Accountability Office released rep...,Government Accountability Office released rep...,,,,,,,,,,,,
9158,Editor note Amy Gahran writes mobile tech CNN ...,Crowdsourced maps help mobile users compare ne...,wireless network maps data reliability strengt...,As about application runs background monitorin...,Amy Gahran CNN . com San Francisco Bay Area Co...,Editor note Amy Gahran writes mobile tech CNN ...,Editor note Amy Gahran writes mobile tech CNN .,Editor note Amy Gahran writes mobile tech CNN ...,,,,,,,,,,,,


In [41]:
jsonFile = open(os.path.join(path_processed_data, 'cnn_train_v3.json'), "r")
jsonContent = jsonFile.read()
details_dict = json.loads(jsonContent)
jsonFile.close()

text_train = details_dict['text']
headlines_train = details_dict['headlines']
keywords_train = details_dict['keywords']

jsonFile = open(os.path.join(path_processed_data, 'cnn_valid_v3.json'), "r")
jsonContent = jsonFile.read()
details_dict = json.loads(jsonContent)
jsonFile.close()

text_valid = details_dict['text']
headlines_valid = details_dict['headlines']
keywords_valid = details_dict['keywords']

jsonFile = open(os.path.join(path_processed_data, 'cnn_test_v3.json'), "r")
jsonContent = jsonFile.read()
details_dict = json.loads(jsonContent)
jsonFile.close()

X_test = details_dict['text']
headlines_test = details_dict['headlines']
keywords_test = details_dict['keywords']

In [42]:
text_test[0:1]

['Story highlights Drummer Lee Riggers Rigby 25 machine gunner became recruiter He real character infantry commanding officer says As drummer stood outside Royal Palaces He also father 2yearold son Jack The British soldier slain gruesome cleaver attack London wellliked infantryman machine gunner served Afghanistan Cyprus became military recruiter ceremonial drummer outside royal palaces military said Thursday .  Drummer Lee Rigby 25 part Regimental Recruiting Team London machine gunner part 2nd Battalion Royal Regiment Fusiliers .  The Fusiliers infantry group known hackle feather plume military headdress .  Rigby 2yearold son Jack UK Ministry Defense said .  Witness Attackers were animals Fellow soldiers described engaging personality .  He joined army 2006 acquired nickname Riggers platoon .  He real character within Second Fusiliers Lt .  Col .  Jim Taylor commanding officer 2nd Fusiliers said statement .  Larger life heart Corps Drums .  An experienced talented side drummer machine

In [43]:
headlines_test[0:1]

['Soldier slain London street machine gunner drummer dad']

In [44]:
keywords_test[0:1]

['street fusiliers member royal soldiers soldier slain gunner drummer machine military dad real london drums']